# 수집된 뉴스 기사 및 댓글에 대한 감정 분석
## * Doc2Vec 
* 데이터 
> 2017년 12월 1일부터 2018년 2월 1일까지 63일간 [네이버](http://www.naver.com)와 [다음](http://www.daum.net)의 랭킹뉴스와 뉴스의 댓글을 크롤링함.

In [ ]:
import pickle
import html
import multiprocessing
from collections import namedtuple, OrderedDict
import re
import sys
import os
from glob import glob
import warnings

os.environ['KERAS_BACKEND']='tensorflow'

from numba import jit
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import pandas as pd

from gensim.models import doc2vec, KeyedVectors
from gensim.models.doc2vec import TaggedDocument

from konlpy.utils import pprint

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, roc_curve,  accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale, MinMaxScaler
from sklearn.manifold import TSNE

import keras.backend.tensorflow_backend as K
from keras.preprocessing import sequence
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers import Input, Flatten, Dense, Embedding, embeddings, merge, Dropout, Activation,  LSTM, Bidirectional, SimpleRNN, GRU
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import SpatialDropout1D
from keras.utils import np_utils
from tensorflow.python.client import device_lib
from keras.layers.merge import dot

import xgboost as xgb

import matplotlib.pyplot as plt

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

In [ ]:
import Database_Handler as dh
import Basic_Module as bm

In [ ]:
import dask.dataframe as dd

In [ ]:
from ckonlpy.tag import Twitter
from konlpy.tag import Mecab
ct = Twitter()
mecab = Mecab()

## Stopwords

In [ ]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

## TaggedDocument For Doc2Vec

In [ ]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags sentiment')

## Load Data

#### Path

In [ ]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/model/'
    classifierPath = '/Volumes/disk1/data/pre_data/classifier/'
    news_senti_outcome = '/Volumes/disk1/outcome_for_News_sentiment_analysis/'
    daumCommentsPath = '/Volumes/disk1/data/daum_Comments/'
    naverCommentsPath = '/Volumes/disk1/data/naver_Comments/'
    outcomeDaumCommentsPath = '/Volumes/disk1/outcome_comments_for_daum/'
    outcomeNaverCommentsPath = '/Volumes/disk1/outcome_comments_for_naver/'
    outcome_predata = '/Volumes/disk1/pre_data_for_comments/'
    outcome_tagged_data = '/Volumes/disk1/pre_data_for_comments2/'
elif sys.platform =='win32':
    loadModelPath = 'd:/model/'
    classifierPath = 'd:/data/pre_data/classifier/'
    newsPath = './data/pre_data/news_sentiment/'
    news_senti_outcome = './outcome_for_News_sentiment_analysis/'
    daumCommentsPath = 'd:/data/daum_Comments/'
    naverCommentsPath = 'd:/data/naver_Comments/'
    outcomeDaumCommentsPath = 'd:/outcome_comments_for_daum/'
    outcomeNaverCommentsPath = 'd:/outcome_comments_for_naver/'
    outcome_predata = 'd:/pre_data_for_comments/'
    outcome_tagged_data = 'd:/pre_data_for_comments2/'

#### News 

In [ ]:
os.listdir(news_senti_outcome)

In [ ]:
# Naver
naverData = pd.read_csv(os.path.join(news_senti_outcome, 'naver_news_sentiment_analysis.csv'), index_col=0, header= 0, encoding = 'utf-8')
naverData['site'] = ['Naver'] * naverData.shape[0]
reNaverData = naverData[naverData.number_of_crawled_comment != 0]
print (reNaverData.shape)
reNaverData.head()

In [ ]:
# Daum
daumData = pd.read_csv(os.path.join(news_senti_outcome, 'daum_news_sentiment_analysis.csv'), index_col=0, header= 0, encoding = 'utf-8')
daumData['site'] = ['daum'] * daumData.shape[0]
reDaumData = daumData[daumData.number_of_crawled_comment != 0]
print (reDaumData.shape)
reDaumData.head()

### 댓글 파일

In [ ]:
os.listdir(outcome_predata)

In [ ]:
%%time
predata_naver = outcome_predata +  'filtered_predata_for_naver_news_comment.csv'
dfNaver = pd.read_csv(predata_naver, header = 0, encoding = 'utf-8')
print (dfNaver.shape)

In [ ]:
%%time
tagged_by_ct_naver_file = outcome_tagged_data+'tagged_data_by_ct_for_naver_news_comment.pickled'
if os.path.isfile(tagged_by_ct_naver_file):
    tagged_naver_ct = pickle.load(open(tagged_by_ct_naver_file, 'rb'))
else:
    tagged_naver_ct = [bm.TokenizeAndTag(ct, dfNaver.loc[idx], stopwords, TaggedDocument) for idx in tqdm(dfNaver.index)]
    pickle.dump(open(tagged_by_ct_naver_file, 'wb'))

In [ ]:
%%time
tagged_by_mecab_naver_file = outcome_tagged_data+'tagged_data_by_mecab_for_naver_news_comment.pickled'
if os.path.isfile(tagged_by_mecab_naver_file):
    tagged_naver_mecab = pickle.load(open(tagged_by_mecab_naver_file, 'rb'))
else:
    tagged_naver_mecab = [bm.TokenizeAndTag(mecab, dfNaver.loc[idx], stopwords, TaggedDocument) for idx in tqdm(dfNaver.index)]
    pickle.dump(open(tagged_by_mecab_naver_file, 'wb'))

In [ ]:
%%time
predata_daum = outcome_predata +  'filtered_predata_for_daum_news_comment.csv'
dfDaum = pd.read_csv(predata_daum, header = 0, encoding = 'utf-8')
print (dfDaum.shape)

In [ ]:
%%time
tagged_by_ct_daum_file = outcome_tagged_data+'tagged_data_by_ct_for_daum_news_comment.pickled'
if os.path.isfile(tagged_by_ct_daum_file):
    tagged_daum_ct = pickle.load(open(tagged_by_ct_daum_file, 'rb'))
else:
    tagged_daum_ct = [bm.TokenizeAndTag(ct, dfDaum.loc[idx], stopwords, TaggedDocument) for idx in tqdm(dfDaum.index)]
    pickle.dump(open(tagged_by_ct_daum_file, 'wb'))

In [ ]:
%%time
tagged_by_mecab_daum_file = outcome_tagged_data+'tagged_data_by_mecab_for_daum_news_comment.pickled'
if os.path.isfile(tagged_by_mecab_daum_file):
    tagged_daum_mecab = pickle.load(open(tagged_by_mecab_daum_file, 'rb'))
else:
    tagged_daum_mecab = [bm.TokenizeAndTag(mecab, dfDaum.loc[idx], stopwords, TaggedDocument) for idx in tqdm(dfDaum.index)]
    pickle.dump(open(tagged_by_mecab_daum_file, 'wb'))

### Twitter & Mecab
> * 메모리, 시간상의 문제점
>> * 메모리는 dataframe apply로 하면 kernel이 죽어버림.
>> * 시간 for 문으로 할 수 있는건 한번에 해버리는게 낫겠지? 

In [ ]:
taggerName_ct = 'ct'
print ('{}'.format(taggerName_ct))
model1_ct = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-10_negative-7_hs-0_dm-1_dm_concat-0_dm_mean-1_by-ct.model')
name1_ct = '-'.join(re.split('[\(\),\/]',str(model1_ct)))+taggerName_ct
print ('Load model')
model2_ct = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-5_negative-7_hs-0_dm-1_dm_concat-1_dm_mean-0_by-ct.model')
name2_ct = '-'.join(re.split('[\(\),\/]',str(model2_ct)))+taggerName_ct
print ('Load model')
model3_ct = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-None_negative-7_hs-0_dm-0_dm_concat-0_dm_mean-0_by-ct.model')
name3_ct = '-'.join(re.split('[\(\),\/]',str(model3_ct)))+taggerName_ct
print ('Load model')
taggerName_mecab = 'mecab'
print ('{}'.format(taggerName_mecab))
model1_mecab = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-10_negative-7_hs-0_dm-1_dm_concat-0_dm_mean-1_by-mecab.model')
name1_mecab = '-'.join(re.split('[\(\),\/]',str(model1_mecab)))+taggerName_mecab
print ('Load model')
model2_mecab = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-5_negative-7_hs-0_dm-1_dm_concat-1_dm_mean-0_by-mecab.model')
name2_mecab = '-'.join(re.split('[\(\),\/]',str(model2_mecab)))+taggerName_mecab
print ('Load model')
model3_mecab = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-None_negative-7_hs-0_dm-0_dm_concat-0_dm_mean-0_by-mecab.model')
name3_mecab = '-'.join(re.split('[\(\),\/]',str(model3_mecab)))+taggerName_mecab
print ('Load model')

In [ ]:
%%time
for idx in reNaverData.index:
    commentFile = os.path.join(naverCommentsPath, idx+'.csv')
    '''
    if os.path.isfile(commentFile):
        print (idx)
        data = reNaverData.loc[idx]
        ## model 1 ct
        bm.PipeLineForSentimentAnalysis(naverCommentsPath, classifierPath, outcomeNaverCommentsPath, data, ct, stopwords, TaggedDocument, model1_ct, name1_ct)
        ## model 2 ct
        bm.PipeLineForSentimentAnalysis(naverCommentsPath, classifierPath, outcomeNaverCommentsPath, data, ct, stopwords, TaggedDocument, model2_ct, name2_ct)
        ## model 3 ct
        bm.PipeLineForSentimentAnalysis(naverCommentsPath, classifierPath, outcomeNaverCommentsPath, data, ct, stopwords, TaggedDocument, model3_ct, name3_ct)  

        ## model 1 mecab
        bm.PipeLineForSentimentAnalysis(naverCommentsPath, classifierPath, outcomeNaverCommentsPath, data, mecab, stopwords, TaggedDocument, model1_mecab, name1_mecab)
        ## model 2 mecab
        bm.PipeLineForSentimentAnalysis(naverCommentsPath, classifierPath, outcomeNaverCommentsPath, data, mecab, stopwords, TaggedDocument, model2_mecab, name2_mecab)
        ## model 3 mecab
        bm.PipeLineForSentimentAnalysis(naverCommentsPath, classifierPath, outcomeNaverCommentsPath, data, mecab, stopwords, TaggedDocument, model3_mecab, name3_mecab)  
    '''

In [ ]:
%%time
for idx in reDaumData.index:
    commentFile = os.path.join(daumCommentsPath, idx+'.csv')
    if os.path.isfile(commentFile):
        if not os.path.isdir(os.path.join(daumCommentsPath, idx)):
            data = reDaumData.loc[idx]
            ## model 1 ct
            bm.PipeLineForSentimentAnalysis(daumCommentsPath, classifierPath, outcomeDaumCommentsPath, data, ct, stopwords, TaggedDocument, model1_ct, name1_ct)
            ## model 2 ct
            bm.PipeLineForSentimentAnalysis(daumCommentsPath, classifierPath, outcomeDaumCommentsPath, data, ct, stopwords, TaggedDocument, model2_ct, name2_ct)
            ## model 3 ct
            bm.PipeLineForSentimentAnalysis(daumCommentsPath, classifierPath, outcomeDaumCommentsPath, data, ct, stopwords, TaggedDocument, model3_ct, name3_ct)  

            ## model 1 mecab
            bm.PipeLineForSentimentAnalysis(daumCommentsPath, classifierPath, outcomeDaumCommentsPath, data, mecab, stopwords, TaggedDocument, model1_mecab, name1_mecab)
            ## model 2 mecab
            bm.PipeLineForSentimentAnalysis(daumCommentsPath, classifierPath, outcomeDaumCommentsPath, data, mecab, stopwords, TaggedDocument, model2_mecab, name2_mecab)
            ## model 3 mecab
            bm.PipeLineForSentimentAnalysis(daumCommentsPath, classifierPath, outcomeDaumCommentsPath, data, mecab, stopwords, TaggedDocument, model3_mecab, name3_mecab)  